# Having fun with DeepBind

In [ ]:
import warnings; warnings.filterwarnings('ignore', category=FutureWarning)
import tensorflow as tf; tf.logging.set_verbosity(tf.logging.ERROR)  # suppress deprecation messages
from depiction.models.examples.deepbind.deepbind import DeepBind, create_DNA_language
from depiction.interpreters.u_wash.u_washer import UWasher
from ipywidgets import interact

## Setup task

In [ ]:
class_names = ['NOT BINDING', 'BINDING']
classifier = DeepBind(model='DeepBind/Homo_sapiens/TF/D00328.003_SELEX_CTCF', min_length=35)
# this class has task (classification) and data_type (text) and some processing defined for your convenience

http://kipoi.org/models/DeepBind/Homo_sapiens/TF/D00328.003_SELEX_CTCF/

# Interpreter parametrization

In [ ]:
# a LIME text interpreter
lime_explanation_configs = {
    'labels': (1,),
}
lime_params = {
    'class_names': class_names,
    'split_expression': list,
    'bow': False,
    'char_level': True
}

# an Anchor text intepreter
anchors_explanation_configs = {
    'use_proba': False,
    'batch_size': 100
}
anchors_params = {
    'class_names': class_names,
    'nlp': create_DNA_language(),
    'unk_token': 'N',
    'sep_token': '',
    'use_unk_distribution': True
}

### Wrapper for the interactive widget

In [ ]:
class InteractiveWrapper:
    def __init__(self, classifier):
        self.classifier = classifier
        self.lime_explainer = UWasher("lime", self.classifier, **lime_params)
        self.anchor_explainer = UWasher("anchors", self.classifier, **anchors_params)

    def callback(self, sequence):
        # LIME
        self.classifier.use_labels = False
        self.lime_explainer.interpret(sequence, explanation_configs=lime_explanation_configs)
        # Anchors 
        self.classifier.use_labels = True
        self.anchor_explainer.interpret(sequence, explanation_configs=anchors_explanation_configs)

# Let's interpret

In [ ]:
wrapper = InteractiveWrapper(classifier)

interact(
    wrapper.callback,
    sequence=[
        'AGGCTAGCTAGGGGCGCCC', 'AGGCTAGCTAGGGGCGCTT', 'AGGGTAGCTAGGGGCGCTT',
        'AGGGTAGCTGGGGGCGCTT', 'AGGCTAGGTGGGGGCGCTT', 'AGGCTCGGTGGGGGCGCTT',
        'AGGCTCGGTAGGGGGCGATT'
    ]
)

CTCF binding motif
![CTCF binding motif](https://media.springernature.com/full/springer-static/image/art%3A10.1186%2Fgb-2009-10-11-r131/MediaObjects/13059_2009_Article_2281_Fig2_HTML.jpg?as=webp)
from Essien, Kobby, et al. "CTCF binding site classes exhibit distinct evolutionary, genomic, epigenomic and transcriptomic features." Genome biology 10.11 (2009): R131.

In [ ]:
classifier_foxa1 = DeepBind('DeepBind/Homo_sapiens/TF/D00761.001_ChIP-seq_FOXA1', min_length=40)

In [ ]:
wrapper_foxa1 = InteractiveWrapper(classifier_foxa1)

interact(wrapper_foxa1.callback, sequence='TGTGTGTGTG')

FOXA1 binding motif
![FOXA1 binding motif](https://ismara.unibas.ch/supp/dataset1_IBM_v2/ismara_report/logos/FOXA1.png)
from https://ismara.unibas.ch/supp/dataset1_IBM_v2/ismara_report/pages/FOXA1.html